In [ ]:
import thunder
from thunder.callbacks import FinetuneEncoderDecoder
from thunder.data.dataset import BaseSpeechDataset, AudioFileLoader
from thunder.data.datamodule import BaseDataModule
from typing import Any, List, Sequence, Tuple, Union
import torch
from torch import Tensor, nn
from thunder.registry import load_pretrained
from thunder.quartznet.compatibility import QuartznetCheckpoint
from pathlib import Path
from thunder.finetune import FinetuneCTCModule
from thunder.blocks import conv1d_decoder
import pytorch_lightning as pl
from IPython.display import Audio

In [ ]:
class TimitDataset(BaseSpeechDataset):
  def open_audio(self, item) -> Tuple[Tensor, int]:
    return self.loader.open_audio(item.Path)
  def open_text(self, item) -> str:
    return item.Transcription
  def get_item(self, index: int) -> Any:
    return self.items.iloc[index]

class TimtiDataModule(BaseDataModule):
  def get_dataset(self, split):
    if split != "train":
      return TimitDataset(labels[labels["is_valid"]])
    else:
      return TimitDataset(labels[labels["is_valid"] == False])

In [ ]:
labels = pd.read_csv('pronunciation-trainer/dataDS.csv')

In [ ]:
whole = ''.join([t for t in labels.Transcription])
tokens = list(set(whole))

In [ ]:
model = FinetuneCTCModule(QuartznetCheckpoint.QuartzNet15x5Base_En,
                          decoder_class = conv1d_decoder, tokens = tokens)

In [ ]:
trainer = pl.Trainer(
    gpus=-1, # Use all gpus
    max_epochs=10,
    callbacks=[FinetuneEncoderDecoder(unfreeze_encoder_at_epoch=1)],
)

In [ ]:
trainer.fit(model = model, datamodule=dm)